<a href="https://colab.research.google.com/github/KimRass/Programming/blob/master/Data%20Science/Machine%20Learning/Natural%20Language%20Preprocessing/Named_Entity_Recognition(SKT_KoBERT_NLP_Challenge_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML, display
def set_css():
  display(HTML("""
  <style>
    pre {white-space: pre-wrap;}
  </style>
  """))
get_ipython().events.register("pre_run_cell", set_css)

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
import sys
try:
    my_path = "/content/notebooks"
    os.symlink("/content/drive/MyDrive/ColabNotebooks/my_env", my_path)
    sys.path.insert(0, my_path)
except:
    pass
os.chdir(my_path)
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Bidirectional,\
TimeDistributed, Flatten, Dropout
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd
from transformers import BertModel
from transformers import TFBertModel
from tokenization_kobert import KoBertTokenizer
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import re
from sklearn.model_selection import train_test_split
import wget

tqdm.pandas()

Mounted at /content/drive


In [ ]:
# !wget https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data

In [ ]:
with open("train_data", "r", encoding="utf-8") as f:
    idx_word_tag = [line.split("\t") for line in f.read().splitlines()]
idx_word_tag = [pair for pair in idx_word_tag if len(pair)==3]

In [ ]:
model = BertModel.from_pretrained("monologg/kobert")
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

In [ ]:
columns = ["order", "word", "tag"]
train_data = pd.DataFrame(data=idx_word_tag, columns=columns)

train_data["order"] = train_data["order"].astype("int")
train_data["word"] = train_data["word"].astype("str")
train_data["tag"] = train_data["tag"].astype("str")

train_data["token"] = train_data["word"].progress_apply(lambda x:tokenizer.tokenize(x))

  0%|          | 0/1063571 [00:00<?, ?it/s]

In [ ]:
tags = set(train_data["tag"])
tag2idx = {tag:idx+1 for idx, tag in enumerate(tags)}
tag2idx["[PAD]"] = len(tag2idx)
idx2tag = {idx:tag for tag, idx in tag2idx.items()}

train_data["idx"] = train_data["tag"].map(tag2idx)

- 중간에 건너뛰는 경우(ex. 1 --> 4)가 없는지 확인합니다.

In [ ]:
num_bef = 0
num = 0
for i, row in tqdm(train_data.iterrows()):
    num_aft = row["order"]
    if num_aft == num_bef + 1:
        pass
    elif num_aft == 1:
        pass
    else:
        num += 1
        print(num)
    num_bef = num_aft

|          | 0/? [00:00<?, ?it/s]

In [ ]:
train_data.head()

,order,word,tag,token,idx
0,1,비토리오,PER_B,"[▁비, 토, 리, 오]",5
1,2,양일,DAT_B,"[▁양, 일]",6
2,3,만에,-,[▁만에],3
3,4,영사관,ORG_B,"[▁영, 사, 관]",9
4,5,감호,CVL_B,"[▁감, 호]",17


- BERT의 입력 형식에 맞추기 전 첫번째 샘플
    - 문장 : `['[CLS]', '비토리오', '양일', '만에', '영사관', '감호', '용퇴', '항룡' '압력설', '의심만', '가율', '[SEP]']`
    - 레이블 : `[2, 0, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2]`
- BERT의 입력 형식에 맞춘 후 첫번째 샘플
    - 문장 : `['[CLS]', '▁비', '토', '리', '오', '▁양', '일', '▁만에', '▁영', '사', '관', '▁감', '호', '▁용', '퇴', '▁항', '룡', '▁압력', '설', '▁의심', '만', '▁', '가', '율', '[SEP]']`  
    - 레이블 : `[2, 0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]`
- 기존의 한 단어였던 것이 BERT 입력 형식에 맞추도록 서브토크나이징 하는 단계에서 여러 서브워드로 나누어지기 때문에 레이블 또한 서브워드 개수만큼 확장해주어야 한다.

In [ ]:
train_data["tags"] = train_data.progress_apply(lambda x:[x["tag"]]*len(x["token"]), axis=1)
train_data["idxs"] = train_data.progress_apply(lambda x:[x["idx"]]*len(x["token"]), axis=1)

  0%|          | 0/1063571 [00:00<?, ?it/s]

  0%|          | 0/1063571 [00:00<?, ?it/s]

In [ ]:
tokens_lists = list()
idxs_lists = list()
temp1 = [train_data.loc[0, "token"]]
temp2 = [train_data.loc[0, "idxs"]]
for _, row in tqdm(train_data[1:].iterrows()):
    if row["order"] != 1:
        temp1.append(row["token"])
        temp2.append(row["idxs"])
    else:
        tokens_lists.append(["[CLS]"] + sum(temp1, []) + ["[SEP]"])
        idxs_lists.append([2] + sum(temp2, []) + [2])
        temp1 = list()
        temp2 = list()

|          | 0/? [00:00<?, ?it/s]

In [ ]:
lens = [len(tokens) for tokens in tokens_lists]
ratio = 0.975
max_len = int(np.quantile(lens, ratio))
print(f"가장 긴 문장의 길이는 {np.max(lens)}입니다.")
print(f"길이가 {max_len} 이하인 문장이 전체의 {ratio:.1%}를 차지합니다.")

가장 긴 문장의 길이는 859입니다.
길이가 82 이하인 문장이 전체의 97.5%를 차지합니다.


- 버트 인풋으로는
    - tokens : 문장이 토크나이즈 된 것이 숫자로 바뀐 것,
    - masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹
    - `[tokens, masks]`가 인풋으로 들어갑니다

In [ ]:
train_val_tokens = pad_sequences([tokenizer.convert_tokens_to_ids(tokens) for tokens in tokens_lists], 
                        maxlen=max_len, value=tokenizer.convert_tokens_to_ids("[PAD]"),
                        truncating="post", padding="post")
train_val_labels = pad_sequences(idxs_lists, maxlen=max_len, value=tag2idx["[PAD]"],
                        padding="post", truncating="post")
train_val_masks = np.array([[1 if token != 1 else 0 for token in subset]\
                  for subset in tqdm(train_val_X)])

  0%|          | 0/89999 [00:00<?, ?it/s]

In [ ]:
train_tokens, val_tokens, train_masks, val_masks, train_labels, val_labels\
 = train_test_split(train_val_tokens, train_val_masks, train_val_labels,
                    train_size=0.8, shuffle=False, random_state=777)

훈련 데이터와 테스트 데이터의 어텐션 마스크를 나눈다. (레이블은 어텐션 마스크가 필요없다.)

### Modeling

In [ ]:
def create_model():
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True,
                                        num_labels=len(tag2idx), output_attentions=False,
                                        output_hidden_states=False)
    input_tokens = Input(shape=(max_len,), name="input_tokens", dtype=tf.int32)
    input_masks = Input(shape=(max_len,), name="input_masks", dtype=tf.int32)
    bert_outputs = model([input_tokens, input_masks])
    bert_outputs
    dense = Dense(units=len(tag2idx), activation="softmax")(bert_outputs[0])
    ner_model = Model(inputs=[input_tokens, input_masks], outputs=dense)

    ner_model.summary()

    ner_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
                      metrics=["sparse_categorical_accuracy"])
    return ner_model

In [ ]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ["COLAB_TPU_ADDR"])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
# strategy = tf.distribute.experimental.TPUStrategy(resolver)
# with strategy.scope():
ner_model = create_model()

# es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=1)
# model_path = "sktkobert_nlpchallenge.h5"
# mc = ModelCheckpoint(filepath=model_path, monitor="sparse_categorical_accuracy", mode="auto",
                    #  verbose=1, save_best_only=True)
his = ner_model.fit(x=[train_tokens, train_masks], y=train_labels,
                    validation_data=([val_tokens, val_masks], val_labels), batch_size=1024,
                    epochs=5, verbose=1, shuffle=True)

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_tokens (InputLayer)       [(None, 82)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 82)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_17 (TFBertModel)  ((None, 82, 768), (N 92186880    input_tokens[0][0]               
                                                                 input_masks[0][0]                
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 82, 30)       23070       tf_bert_model_17[0][0]    

71/71 [==============================] - 3224s 45s/step - loss: 1.9477 - sparse_categorical_accuracy: 0.5468 - val_loss: 1.3010 - val_sparse_categorical_accuracy: 0.6402
Epoch 2/5
18/71 [======>.......................] - ETA: 37:27 - loss: 1.2882 - sparse_categorical_accuracy: 0.6462

UnavailableError: ignored

In [ ]:
model.save("sktkobert_nlpchallenge.h5")

NameError: ignored

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [ ]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

In [ ]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       PER_B       0.88      0.87      0.87     11330
       DAT_B       0.90      0.84      0.87      4239
           -       0.95      0.91      0.93    127803
       ORG_B       0.89      0.82      0.85     11616
       CVL_B       0.79      0.85      0.82     14438
       NUM_B       0.95      0.91      0.93     10411
       LOC_B       0.86      0.82      0.84      5621
       EVT_B       0.78      0.81      0.80      3122
       TRM_B       0.86      0.80      0.83      6993
       TRM_I       0.51      0.42      0.46       681
       EVT_I       0.82      0.74      0.78      1545
       PER_I       0.72      0.74      0.73      1573
       CVL_I       0.48      0.46      0.47       784
       NUM_I       0.67      0.81      0.73      1461
       TIM_B       0.84      0.89      0.86       547
       TIM_I       0.90      0.87      0.88       224
       ORG_I       0.63      0.83      0.71      1326
       DAT_I       0.85    

F1-score에서 89% 달성

In [ ]:

def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, pad_to_max_length=True)])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
  
    if (token.stokentswith("▁")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[1:])
    elif (token=="[CLS]"):
      pass
    elif (token=="[SEP]"):
      pass
    elif (token=="[PAD]"):
      pass
    elif (token != "[CLS]" or token != "[SEP]"):
      new_tokens[-1] = new_tokens[-1] + token

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [ ]:
ner_inference("문재인 대통령은 1953년 1월 24일 경상남도 거제시에서 아버지 문용형과 어머니 강한옥 사이에서 둘째(장남)로 태어났다.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


PER_B	문재인
CVL_B	대통령은
DAT_B	1953년
DAT_I	1월
DAT_I	24일
LOC_B	경상남도
LOC_B	거제시에서
CVL_B	아버지
PER_B	문용형과
CVL_B	어머니
PER_B	강한옥
-	사이에서
NUM_B	둘째(장남)로
-	태어났다.


In [ ]:
ner_inference("9세이브로 구완 30위인 LG 박찬형은 평균자책점이 16.45로 준수한 편이지만 22이닝 동안 피홈런이 31개나 된다.")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


NUM_B	9세이브로
-	구완
NUM_B	30위인
ORG_B	LG
PER_B	박찬형은
-	평균자책점이
NUM_I	16.45로
-	준수한
-	편이지만
NUM_B	22이닝
-	동안
NUM_B	피홈런이
NUM_B	31개나
-	된다.


In [ ]:
ner_inference("인공지능의 역사는 20세기 초반에서 더 거슬러 올라가보면 이미 17~18세기부터 태동하고 있었지만 이때는 인공지능 그 자체보다는 뇌와 마음의 관계에 관한 철학적인 논쟁 수준에 머무르고 있었다. 그럴 수 밖에 없는 것이 당시에는 인간의 뇌 말고는 정보처리기계가 존재하지 않았기 때문이다. ")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


TRM_B	인공지능의
-	역사는
NUM_B	20세기
-	초반에서
-	더
-	거슬러
-	올라가보면
-	이미
NUM_B	17~18세기부터
-	태동하고
-	있었지만
-	이때는
TRM_B	인공지능
-	그
-	자체보다는
-	뇌와
-	마음의
-	관계에
-	관한
-	철학적인
-	논쟁
-	수준에
-	머무르고
-	있었다.
-	그럴
-	수
-	밖에
-	없는
-	것이
-	당시에는
-	인간의
-	뇌
-	말고는
TRM_B	정보처리기계가
-	존재하지
-	않았기
-	때문이다.
